# Projet maison n° 3

In [3]:
# imports
import pandas as pd
import glob
import os

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [4]:
# names us
def df_names_us():
    df = pd.DataFrame()
    path = r'C:\Users\evawg\Kit-Big-Data\session3\names'
    all_files = glob.glob(os.path.join(path, "*.txt"))
    li = []
    for f in all_files:
        df_temp = pd.read_csv(f, names = ['name', 'gender', 'births'] )
        df_temp['year'] = os.path.basename(f)[-8:-4]
        df_temp = df_temp[['year', 'name', 'gender', 'births']]
        li.append(df_temp)
    df = pd.concat(li, ignore_index=True)
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [5]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    path = r'C:\Users\evawg\Kit-Big-Data\session3\nat2021_csv'
    files = glob.glob(os.path.join(path, "*.csv"))
    li = []
    for f in files:
        df_temp = pd.read_csv(f, sep = ";", keep_default_na=False)
        new_cols = {0: "gender", 1: "name", 2: "year", 3: "births"}
        df_temp = df_temp.rename(columns={df_temp.columns[i]: new_col for i, new_col in new_cols.items()})
        df_temp = df_temp[['year', 'name', 'gender', 'births']]
        mapping0 = {'1': 'M', '2': 'F'}
        df_temp['gender'] = df_temp['gender'].apply(lambda x: str(x)).map(mapping0)
        df_temp = df_temp[df_temp['name'].apply(lambda x: str(x)).map(len) >= 2]
        df_temp['name'] = df_temp['name'].str.title()
        df_temp.drop(df_temp.loc[df_temp['name'] == '_Prenoms_Rares'].index, inplace = True)
        df_temp.drop(df_temp.loc[df_temp['year'] == 'XXXX'].index, inplace = True)
        df_temp.dropna(axis = 0, inplace = True)
        df_temp.sort_values(by =['year', 'gender', 'births', 'name'], inplace = True, ascending = [True, True, False, True])
        df_temp.reset_index(inplace = True, drop = True)
        li.append(df_temp)
    df = pd.concat(li, ignore_index=True)    
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [123]:
# taux de change
import re
import numpy as np
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export.csv', sep = ';', skiprows = range(1,6))
    df = df.set_index('Titre :')
    new_cols = re.findall('\((.*?)\)',str(df.iloc[0]))
    indexes = list(range(len(new_cols)))
    col_names = dict(zip(indexes, new_cols))
    df = df.rename(columns={df.columns[i]: col_name for i, col_name in col_names.items()})
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.apply(lambda x: x.replace({'-': None}))
    df.dropna(axis = 0, inplace = True)
    df = df.astype(float)
    df.index = pd.to_datetime(df.index, format = "%d/%m/%Y")
    df = df.sort_index(ascending = True)
    return df[devise]

### Tests

In [124]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [125]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [126]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 10.489s

OK
